# Using Amazon Machine Image with Preloaded Theano

## Setting up with Amazon EC2

A few links to [Amazon EC2 Documentation](http://aws.amazon.com/documentation/ec2/) are provided here to help you set up with Amazon EC2. If you have already set up with Amazon EC2, you can skip to the next section.

- [Amazon student](https://aws.amazon.com/education/awseducate/) provides $35 credit to students. 
- General information about Amazon EC2 can be found [here](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/concepts.html).
- To sign up for AWS and prepare for launching an instance, follow the documentation [here](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/get-set-up-for-amazon-ec2.html).
- To start an Amazon EC2 Linux instance, follow [these steps](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html).
- Make sure to check out these [best practice guides](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-best-practices.html) to better manage your instances.
- Make sure that you understand the [pricing information](https://aws.amazon.com/ec2/pricing/) for EC2 instances and storage.

If you follow these steps, the setup time usually takes 1-2 hours. Here are more detailed steps after you have signed up for Amazon Web Service.

### Creating a Group and a User

Following [Create an IAM User](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/get-set-up-for-amazon-ec2.html#create-an-iam-user) Section exactly to create an administrative user. At the end of the section, you should have created a user and log into AWS console using AIM Users Sign-in Link.

### Create a Security Group

Choose EC2 from Amazon Web Services and you will be taken to your EC2 console. Make sure you choose US East (N Virginia) as your region on the upper right corner. You will not be able to access the provided machine image from any other regions.

Navigate to Security Groups and click Create Security Group button. Add the following rules for your inbound rules, and choose my IP for the last two roles. The ssh port will allow you to log in through terminal, and the 9999 port is needed to access your Jupyter notebook server.
<img src='files/files/sg.png' />

### Creating a Key Pair

Navigate to Key Pairs in your EC2 console, and click Create Key Pair. Type in a name for the key.
<img src='files/files/key-pair.png' />
A key will then be automatically downloaded to your computer. Save that key to a folder that you will remember.

For Linux/OS X, open a terminal, cd into the folder where you store the key and execute the following

In [ ]:
chmod 400 file-name-of-your-key.pem

For Windows, open PuTTygen.exe and click on load keys, choose all file types and select the key that you downloaded. Then click on genearte private key. Click yes when prompted a warning, and choose a file name that you will store this key. The private key will be stored to a file with extension .ppk.

## Creating an EBS Volume for Storing Files

It is recommended that you create an additional EBS Volume to store your own files apart from the OS volume that will be created when launching the instance. This will be important to ensure that you do not loose your data when the instance is terminated, particularly if you are using Spot Instances. To do so, navigate to Volumes in your EC2 console, and click Create Volume. Enter the size of your volume. Please note how much storage you can get for free tier. Remember to select an availability zone. For example, we choose us-east-1b. When you launch your instance later, you need to choose a subnet for the same zone in order to attach this volume to your instance.
<img src='files/files/create-volume.png' />

## Launching an GPU instance Using Preloaded Amazon Machine Image

We made available a public Amazon Machine Image (AMI) in which all packages required by Theano and Theano itself have been preloaded. 
The AMI is only available in US East Region currently, and you must launch the AMI in this region. Please follow the steps below to launch an GPU instance using the AMI.

First, change your region to US East and search in the public images for E6040_AMI. Click on the one with AMI ID: ami-e3e3c689. Then click Launch button on the top.
<img src='files/files/ec2-ami-1.png' />

You will be prompted to Step 2 of instance setup. 
Here, choose either g2.2xlarge for an instance with 1 GPU or g2.8xlarge for one with 4 GPUs.
<img src='files/files/ec2-launch-step2.png' />

In step 3: Choose the subnet corresponding to the availability zone on which you created the EBS Volume.
<img src='files/files/subnet.png' />
You can also check Request Spot Instances and enter the maximum price that you want to pay per hour. If this is higher than the current price for the availability zone you chose, your instance will be fulfilled. This can significantly lower your cost. Remeber, however, that your instance will be terminated if either the current price is higher than the price you bid, or there are no long any unused EC2 instances available for spot request.

In step 4, add storage with at least 8 GiB size. If you wish to keep the root storage, uncheck "Delete on Termination" box, otherwise leave it checked. Add additional storage as you need.
<img src='files/files/ec2-launch-step4.png' />

In step 5, you can leave it as is or create a new tag for you instance.

In step 6, select an existing security group, and choose the Security Group you have created earlier. Then, review and launch the instance. 

### Accessing your instance

If you are using Linux or OS X, you can log into your instance by

In [ ]:
ssh -i path-to-your-key.pem ubuntu@replace-ec2-DNS-here

For file transfer, you can you sftp:

In [ ]:
sftp -i path-to-your-key.pem ubuntu@replace-ec2-DNS-here

If you are using windows, see [this page](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/putty.html) for information about logging into the system using PuTTY. For file transfer, you can use WinSCP.

### Attaching Additional EBS Volume

Navigate to Instances in your EC2 console. Copy the instance id of your running instance. Navigate to Volumes. Click on the EBS Volume you created earlier for storing files, and choose Attach Volume from Actions. Paste the instance id under Instance ID, and you should see this
<img src='files/files/att-volume.png' />

Log into your running instance. **If you are doing this for the first time**, you need to format the attached hard drive. You should first check if there is a partition on the volume by

In [ ]:
sudo file -s /dev/xvdf

If this returns /dev/xvdf: data, then you need to format it by

In [ ]:
sudo mkfs -t ext4 /dev/xvdf

If the previous comment return a line with UUID, then you must have formatted the drive before.

Every time you start a new instance, you can create a mounting point and mount the volume by

In [ ]:
cd
mkdir extern
sudo mount /dev/xvdf extern
sudo chown ubuntu:ubuntu extern
df -h # check if the drive is mounted correctly.

If the drive is mounted properly, you should see the output of the last command has a line indicating /dev/xvdf with mouting point /home/ubuntu/extern. You can now store all your files in the folder "extern". It is recommended that you only store your files and data on this volume. This way, your files and instances are independent. You can always detach the volume from one instance, terminate that instance, and attach the volume to a new instance later on. To unmount a volume from your instance

In [ ]:
sudo umount /dev/xvdf

You can detach the volume in Volumes in EC2 console by choosing Detach Volume.

## Using Theano

Once you are logged into your instance, actviate the Theano environment by:

In [ ]:
source activate theano

Environment is managed by [Conda](http://conda.pydata.org/docs/) using miniconda2 installed under /home/ubuntu. You can now start using Theano. This is the environment that holds all packages that are needed to execute Theano. Therefore, you need to make sure that the environment is activated before using Theano. If the theano environment is activated, you should see '(theano)' at the begining of each line in terminal.

## Create Your Own AMI

If you install additional software packages on top of the provided AMI, and want to avoid installing them again the next time you launch an instance, you can create an image on your own instance. Details can be found [here](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/creating-an-ami-ebs.html).

## Using Jupyter Notebook on the GPU instance

### Configuring Jupyter Notebook Server

Jupyter has been preloaded on the AMI. To launch a notebook server, follow the [instruction](http://jupyter-notebook.readthedocs.org/en/latest/public_server.html), or do the following:

In [ ]:
source activate theano
ipython

In [ ]:
In [1]: from notebook.auth import passwd
In [2]: passwd()

When prompted, set a password of your choice. **This will be the password that you will use to log into the notebook from your browser.** Copy the output starting with 'sha1:'.

**Exit ipython**. Edit the file ~/.jupyter/jupyter_notebook_config.py, **scroll to the end of the file**, and paste the above output in between the quotes on the line (uncomment the line by removing # at the beginning of the line):

In [ ]:
c.NotebookApp.password = ''

Create an self-assigned certificate to enable secure connection to the server, and make sure that the two files: mykey.key and mycert.pem are created under your home folder /home/ubuntu. (Exit ipython before you execute the following commands)

In [ ]:
cd ~/
openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mykey.key -out mycert.pem

### Using Jupyter Notebook

Launch jupyter notebook

In [ ]:
jupyter notebook

In your EC2 console, add a TCP rule to the security group that your instance is using (if you have not added it in the Create Security Group step).
<img src='files/files/add_port.png' />
You can now access the notebook server from your browser by enter the DNS address of your instance, e.g.: https://ec2-x-x-x-x.amazonaws.com:9999. The default port is set to 9999, this can be changed at the end of the file ~/.jupyter/jupyter_notebook_config.py. Log into the notebook server using the password you created 

## Configuration Files

The following configuration is used in .bashrc file in the AMI (based on [Using GPU](http://deeplearning.net/software/theano/install.html#gpu-linux)):

In [ ]:
export THEANO_FLAGS="cuda.root=/usr/local/cuda,device=gpu,floatX=float32"

You can also manually switch between CPU and GPU before executing python code, see examples [here](http://deeplearning.net/software/theano/tutorial/using_gpu.html).

The following configuration is added to the .jupyter/jupyter_notebook_config.py

In [ ]:
c.NotebookApp.password = ''                     

c.NotebookApp.certfile = '/home/ubuntu/mycert.pem'
c.NotebookApp.keyfile = '/home/ubuntu/mykey.key'

c.NotebookApp.ip = '*'
c.NotebookApp.open_browser = False
                                  
c.NotebookApp.port = 9999